# Test code from ChatGPT


Import data

In [ ]:
import numpy as np
from pathlib import Path

# shape: (n_timepoints, n_rois)
bold_subject = np.load("sub-01_movie_bold_ROI.npy")

data_dir = Path("path/to/roi_timeseries")

subjects = ["sub-01", "sub-02", "sub-03", ...]
ages = {"sub-01": 23, "sub-02": 68, ...}  # or group labels directly
young_subs = [s for s in subjects if ages[s] < 40]
old_subs   = [s for s in subjects if ages[s] > 60]

roi_ts = {}
for sub in subjects:
    roi_ts[sub] = np.load(data_dir / f"{sub}_movie_roi_ts.npy")  
    # roi_ts[sub].shape = (T, N_rois)

Apply MVMD - Multivariate Variational Modal Decomposition

In [ ]:
# Code to apply MVMD
import mvmd


Compute timescale-specific connectivity

In [ ]:
def corr_matrix(X):
    """
    X: array of shape (T, N_rois)
    returns: (N_rois, N_rois) correlation matrix
    """
    # demean
    X = X - X.mean(axis=0, keepdims=True)
    # normalise
    X = X / X.std(axis=0, keepdims=True)
    # correlation as dot product
    return (X.T @ X) / (X.shape[0] - 1)

K = 5
fc_modes = {}        # fc_modes[sub][k] = (N_rois, N_rois)
fc_broadband = {}    # conventional broadband

for sub in subjects:
    X = roi_ts[sub]                  # broadband data
    fc_broadband[sub] = corr_matrix(X)
    
    modes_sub = mvmd_modes[sub]      # (K, T, N_rois)
    fc_modes[sub] = {}
    for k in range(K):
        X_k = modes_sub[k]           # (T, N_rois) for mode k
        fc_modes[sub][k] = corr_matrix(X_k)

Use Hilbert Transform to get instantaneous frequency

In [2]:
from scipy.signal import hilbert

def mode_instantaneous_frequency(mode_ts, TR):
    """
    mode_ts: (T,)  – one ROI, one mode
    TR: repetition time in seconds
    returns: instantaneous frequency array in Hz
    """
    analytic = hilbert(mode_ts)
    phase = np.unwrap(np.angle(analytic))
    inst_freq = np.diff(phase) / (2 * np.pi * TR)
    return inst_freq

def mode_mean_frequency_for_subject(modes_sub, TR):
    """
    modes_sub: (K, T, N_rois)
    returns: (K,) mean frequency per mode (averaged across time and ROIs)
    """
    K, T, N = modes_sub.shape
    freqs = np.zeros(K)
    for k in range(K):
        inst_freq_all = []
        for r in range(N):
            f = mode_instantaneous_frequency(modes_sub[k, :, r], TR)
            inst_freq_all.append(f)
        inst_freq_all = np.concatenate(inst_freq_all)
        freqs[k] = np.nanmean(inst_freq_all)
    return freqs

# Example usage
TR = 2.0  # example
freqs_sub = mode_mean_frequency_for_subject(mvmd_modes["sub-01"], TR)
print(freqs_sub)  # approximate central frequencies in Hz

Group-level comparison: young vs. old

In [ ]:
def vec_from_matrix(M):
    # take upper triangle without diagonal
    idx = np.triu_indices_from(M, k=1)
    return M[idx]

# Build matrices of features: subjects × edges
import pandas as pd

def build_group_matrix(sub_list, k=None):
    # k = None => broadband; otherwise intrinsic mode k
    data = []
    for sub in sub_list:
        if k is None:
            M = fc_broadband[sub]
        else:
            M = fc_modes[sub][k]
        data.append(vec_from_matrix(M))
    return np.vstack(data)

X_young_broad = build_group_matrix(young_subs, k=None)
X_old_broad   = build_group_matrix(old_subs, k=None)

X_young_modes = {k: build_group_matrix(young_subs, k) for k in range(K)}
X_old_modes   = {k: build_group_matrix(old_subs, k) for k in range(K)}

In [ ]:
from scipy.stats import ttest_ind

def edgewise_ttest(X_y, X_o):
    # X_y, X_o: (n_subs_group, n_edges)
    tvals, pvals = ttest_ind(X_y, X_o, axis=0, equal_var=False)
    return tvals, pvals

stats_modes = {}
for k in range(K):
    tvals, pvals = edgewise_ttest(X_young_modes[k], X_old_modes[k])
    stats_modes[k] = {"t": tvals, "p": pvals}
    
tvals_broad, pvals_broad = edgewise_ttest(X_young_broad, X_old_broad)

Correct p for multiple comparisons (FDR/BH) and Reshape signifcant effects back into ROIxROI matrices for plotting

In [ ]:
from statsmodels.stats.multitest import multipletests

def p_to_sig_matrix(pvals, N_rois, alpha=0.05):
    _, p_corr, _, _ = multipletests(pvals, alpha=alpha, method='fdr_bh')
    sig = p_corr < alpha
    M = np.zeros((N_rois, N_rois), dtype=bool)
    idx = np.triu_indices(N_rois, k=1)
    M[idx] = sig
    M = M + M.T
    return M, p_corr

N_rois = roi_ts[subjects[0]].shape[1]
sig_masks = {}

for k in range(K):
    pvals = stats_modes[k]["p"]
    sig_masks[k], p_corr_k = p_to_sig_matrix(pvals, N_rois)
    
sig_mask_broad, p_corr_broad = p_to_sig_matrix(pvals_broad, N_rois)

In [ ]:
for k in range(K):
    n_edges = sig_masks[k].sum() / 2  # undirected graph
    print(f"Mode {k}: {n_edges} significant edges")
    
n_edges_broad = sig_mask_broad.sum() / 2
print(f"Broadband: {n_edges_broad} significant edges")

Compare to conventional band-pass connectivity

In [ ]:
from scipy.signal import butter, filtfilt

def bandpass_filter(ts, low, high, TR, order=2):
    nyq = 0.5 / TR
    b, a = butter(order, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, ts, axis=0)

TR = 2.0
fc_bandpass = {}

for sub in subjects:
    X = roi_ts[sub]
    Xf = bandpass_filter(X, 0.01, 0.1, TR)
    fc_bandpass[sub] = corr_matrix(Xf)